In [1]:
# !pip install transformers
# !pip install accelerate
# TODO : head -> lora, github tutorial 
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [2]:
# #양자화에 필요한 패키지 설치
# !pip install -q -U bitsandbytes
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install git+https://github.com/huggingface/peft


In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingLR

from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
CFG = {
    'LR' : 1e-5, # Learning Rate
    'EPOCHS' : 50, # 학습 Epoch
    'BATCH_SIZE' : 1,
    'AUG_RATIO' : 0.15,
    'AUG_PROB' : 0.5,
}

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
    # load_in_4bit=True,
    # bnb_4bit_use_double_quant=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16

In [6]:
data_df = pd.read_csv('./data/train.csv')
# data_df = data_df.drop(['id','category'], axis=1)


In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("yanolja/KoSOLAR-10.7B-v0.2")
# model = AutoModelForCausalLM.from_pretrained("yanolja/KoSOLAR-10.7B-v0.2")

tokenizer = AutoTokenizer.from_pretrained("./solar_29_epoch/")
model = AutoModelForCausalLM.from_pretrained("./solar_29_epoch",quantization_config=bnb_config) # quantization_config=bnb_config,)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
import json
# Open the JSON file
with open('./rag_data.json', 'r') as json_file:
    # Load the JSON data
    reply_data = json.load(json_file)

reply_data


{'0': ['면진장치는 건축 구조물에서 수직 및 수평으로 나란히 놓여진 구조물을 연결하거나 보강하는 장치를 말합니다. 이 장치는 건물의 안정성을 유지하고 향상시키며 지진이나 바람에 의한 외부 영향을 견디도록 도와줍니다. 보통 철강이나 콘크리트로 만들어지며 건물의 구조적인 안전을 보장하기 위해 중요한 역할을 합니다.',
  '면진장치는 건축물의 안전성을 평가하고 지지력을 유지하기 위한 중요한 요소입니다. 주요 기술로는 다음과 같은 것들이 있습니다:\n\n1. 보강벽: 면진장치 중 가장 일반적인 형태로, 건물 주변에 보강벽을 설치하여 지지력을 강화합니다.\n2. 지지벽: 지지벽은 주변 지형이나 지반의 변화로 발생하는 압력을 분산시키고 구조물을 지탱하는 역할을 합니다.\n3. 지진기술: 지진 발생 시 건물의 흔들림을 최소화하기 위한 기술로, 강도가 높은 지진에도 건물이 붕괴하지 않도록 설계합니다.\n4. 지반조사: 건축물을 건설하기 전에 지반의 특성을 조사하여 적절한 면진장치를 설치하는 것이 중요합니다.\n5. 지진진동흡수기술: 건물 구조물에 지진진동을 흡수하고 분산시키는 기술을 적용하여 건물의 안정성을 높입니다.\n\n이러한 기술들이 면진장치의 안전성과 지지력을 보장하는 데 중요한 역할을 합니다.'],
 '1': ['내진설계는 건물이 지진 발생 시 안전하게 유지될 수 있도록 설계하는 것을 말합니다. 주요 내진설계의 종류는 다음과 같습니다:\n\n1. 단순내진설계: 건물의 구조체를 강화하여 지진에 대비하는 가장 기본적인 내진설계 방법입니다.\n\n2. 강풍내진설계: 바람이나 강풍에 의한 하중에 대비하기 위해 건물을 설계하는 방법으로, 지진 발생 시에도 안전을 보장합니다.\n\n3. 복합내진설계: 지진 및 바람에 대비하여 건물을 설계하는 방법으로, 다양한 하중을 고려하여 안전성을 강화합니다.\n\n4. 토양내진설계: 건물의 기초와 토양 사이의 상호작용을 고려하여 지진에 대비하는 설계 방법입니다.\n\n이외에도 다양한 내진설계 방법이 있을 수 있으며, 건축물의 특성과

In [9]:

list_r_6 = []
list_r_7 = []
for idx, row in reply_data.items():

    list_r_6.append(row[0].replace('\n',' '))
    list_r_7.append(row[1].replace('\n',' '))

data_df['답변_6'] = list_r_6
data_df['답변_7'] = list_r_7



In [10]:
data_df = data_df.drop(['id','category'], axis=1)
data_df

,질문_1,질문_2,답변_1,답변_2,답변_3,답변_4,답변_5,답변_6,답변_7
0,면진장치가 뭐야?,면진장치에 사용되는 주요 기술은 무엇인가요?,면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 ...,"면진장치란 건물의 지반에서 발생하는 진동 에너지를 흡수하여 건물을 보호하고, 진동을...",면진장치란 지반으로부터 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여...,면진장치는 건물의 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 ...,면진장치는 건물에 오는 지반 진동의 영향을 최대한으로 흡수하여 건물에 전달되는 진동...,면진장치는 건축 구조물에서 수직 및 수평으로 나란히 놓여진 구조물을 연결하거나 보강...,면진장치는 건축물의 안전성을 평가하고 지지력을 유지하기 위한 중요한 요소입니다. 주...
1,내진설계의 종류 좀 알려줘,내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요?,"내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다.","내진설계에는 내진구조, 제진구조, 면진구조가 있습니다. 내진구조는 건물 구조물이 지...","내진설계에는 주로 내진구조, 제진구조, 면진구조의 세 가지 종류가 있습니다. 이들은...","내진설계에는 주로 내진구조, 제진구조, 면진구조가 사용됩니다. 내진구조는 건물 구조...","내진 설계에는 다양한 종류가 있지만, 대표적으로 내진구조, 제진구조, 면진구조가 있...",내진설계는 건물이 지진 발생 시 안전하게 유지될 수 있도록 설계하는 것을 말합니다....,"네, 지진에 대비한 건축물의 설계를 의미하는 '내진설계'에는 여러 가지 종류가 있습..."
2,철골구조의 장점이 뭐야?,철골구조의 장점을 알려줘?,철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가...,철골구조의 장점은 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건...,철골구조의 장점은 건물의 외벽에 하중이 적게 걸리기 때문에 고층 건물의 건축이 용이...,"철골구조의 장점은 건물의 외벽이 하중이 걸리지 않아 공간 활용이 용이하고, 고층 건...",철골구조의 장점은 건물의 외벽에 하중이 크게 걸리지 않아 고층 건물을 건축할 수 있...,철골구조의 장점은 다음과 같습니다: 1. 강도: 철골은 강철로 만들어져 있어서 뛰...,철골구조는 건물을 지탱하고 지지하는 데 사용되는 구조물의 한 유형입니다. 이러한 구...
3,철골철근 콘크리트 구조가 뭐야?,철골철근 콘크리트 구조의 장점과 단점에는 무엇이 있을까요?,"철근철골콘크리트는 철골과 철근, 그리고 콘크리트를 함께 사용하는 건축 구조입니다. ...","철골철근콘크리트 구조는 건축물을 지탱하는 주요 구조물인 철골과 철근, 그리고 콘크리...",철골철근 콘크리트 구조는 건축물을 지탱하기 위한 구조물에서 일반적으로 사용되는 방식...,"철골철근콘크리트 구조는 철골과 철근, 그리고 콘크리트를 함께 사용하여 만들어지는 건...","철골철근 콘크리트 구조는 강철 골조와 강철 철근, 그리고 콘크리트를 함께 사용하여 ...",철골철근 콘크리트 구조는 건축물을 지탱하고 지지하는데 사용되는 구조적 시스템입니다....,철골철근 콘크리트 구조의 장단점은 다음과 같습니다. 장점: 1. 강도와 내구성이 ...
4,철골구조는 어떤 방식이 있어?,철골구조의 다양한 방식이 무엇인가요?,철골구조는 일반철골구조와 경량철골구조가 있습니다.,철골구조는 일반철골구조와 경량철골구조가 있습니다. 일반철골구조는 주로 대형 건물이나...,철골구조는 주로 일반철골구조와 경량철골구조로 나뉘어집니다. 이들은 건축 시스템에 따...,철골구조는 주로 일반철골구조와 경량철골구조로 구분됩니다. 이외에도 최근에는 고층 건...,철골구조는 일반철골구조와 경량철골구조 두 가지 방식이 주로 사용됩니다. 일반철골구조...,철골구조는 주로 강철을 사용하여 건물 또는 다른 구조물을 지지하는 구조 방식입니다....,"철골구조는 강철로 만들어진 구조물을 지칭하는 용어로, 다양한 방식으로 설계될 수 있..."
...,...,...,...,...,...,...,...,...,...
639,벽장 부위 결로의 원인이 뭐야?,벽장 부위 결로가 발생하는 주된 원인은 무엇일까요?,벽장 부위 결로의 원인은 난방이 이웃한 방과동일한 조건이 되나 그 방에 비해 저온인...,"벽장 부위 결로의 원인은 주로 난방이 자리잡은 방이 내부 온도가 낮은 반면, 외부 ...",벽장 부위 결로가 발생하는 원인은 난방이 잘 이루어지지 않아 해당 공간이 저온인 반...,"벽장 부위 결로가 발생하는 원인은 난방이 인접한 방과 같은 조건을 갖추고 있지만, ...",벽장 부위 결로의 주된 원인은 충분한 환기가 이루어지지 않는 환경과 과도한 습기가 ...,벽장 부위에 생긴 결로의 주요 원인은 다음과 같을 수 있습니다: 1. 고습한 환경...,벽장 부위 결로가 발생하는 주된 원인은 다음과 같습니다: 1. 습기: 과도한 습기...
640,"AD, PD에 면한 벽체 결로의 원인이 뭐야?",벽체 결로가 AD나 PD에 면한다면 그 원인이 무엇인가요?,"AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 원인은 외기에 접하는 면 좌, 우측 벽체에는 단열재...","AD, PD에 면한 벽체 결로의 주된 원인은 외기에 접하는 면 좌, 우측 벽체에 단...","AD, PD에 면한 벽체 결로의 원인으로는 외기에 접하는 면 좌, 우측 벽체는 일반...","AD, PD에 면한 벽체 결로의 원인으로는 외부 공기에 노출된 벽면이 실내보다 냉각...",벽체 결로는 다양한 요인에 의해 발생할 수 있으나 주로 습기와 환경 조건이 원인이 ...,"벽체 결로가 AD나 PD에 면하는 경우, 이는 공간 내 습도와 온도 조절이 부족하여..."
641,외벽 모서리 부위에 결로가 발생하는 원인이 뭐야?,외벽 모서리 부위에 결로가 발생하는 것을 예방하는 방법이 있을까요?,외벽모서리 부위에 결로가 발생하는 원인은 높은 온도차 때문입니다. 외벽 모서리의 경...,외벽 모서리 부위에 결로가 발생하는 원인은 중요한 역할을 합니다. 외부에 비치는 세...,외벽 모서리 부위에 결로가 발생하는 원인으로는 높은 온도차가 대표적입니다. 외벽 모...,외벽 모서리 부위에 결로가 발생하는 원인은 실내와 외부의 온도차 때문입니다. 외벽 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 높은 온도차입니다. 외벽 모서리는 ...,외벽 모서리 부위에 결로가 발생하는 주요 원인은 주로 습도가 높은 환경과 공기 흐름...,외벽 모서리 부위에 결로가 발생하는 것을 예방하기 위해서는 몇 가지 조치를 취할 수...
642,창호 결로의 대책은 뭐야?,창호 결로를 해결하기 위한 가장 효과적인 방법은 무엇인가요?,창호결로의 대책은 제품을 선정할 때 KS에 규정된 프레임을 선정하고 유리의 열관류율...,창호 결로의 대책은 KS에 규정된 프레임을 선정하고 유리의 열관류율 및 결로 발생 ...,"창호 결로를 예방하기 위한 대책은 KS에 규정된 프레임을 선택하고, 열관류율이 적은...","창호 결로의 대책은 여러 가지가 있습니다. 먼저, 창호를 선택할 때 KS에 규정된 ...","창호 결로를 방지하기 위한 대책으로는 KS에 규정된 프레임을 사용하고, 열관류율 및...","창호 결로는 주로 습기가 많은 환경에서 발생하는 문제로, 실내 환경의 습도를 조절하...",창호 결로는 습기와 곰팡이 성장을 촉진할 수 있는 심각한 문제입니다. 이를 해결하기...


In [11]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40960, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [12]:
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_int8_training
from peft import prepare_model_for_kbit_training

lora_config = LoraConfig(r=1,
                         lora_alpha=2,
                         target_modules=['q_proj','k_proj','v_proj','o_proj'],
                         lora_dropout=0.1,
                         # inference_mode=False,
                         bias="none",
                         task_type=TaskType.CAUSAL_LM)


# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)
peft_model = get_peft_model(model, lora_config)




In [13]:
# from peft import get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit, PeftModel, PeftConfig

# peft_model = PeftModel.from_pretrained(
#     model,    # 프롬프트 튜닝에 사용될 기본 모델
#     "./solar_2_lora_14_epoch/",     # Peft 모델이 저장된 위치
#     is_trainable = True 
# )

In [14]:
peft_model.print_trainable_parameters()
peft_model.to(device)

trainable params: 1,277,952 || all params: 10,806,202,368 || trainable%: 0.011826097240084557


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(40960, 4096)
        (layers): ModuleList(
          (0-47): 48 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=1, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=1, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_

In [15]:
# peft_model

In [16]:
import random


class HansolDataset(Dataset):
    def __init__(self, data_df, tokenizer):
        self.tokenizer = tokenizer
        self.data_df = data_df
        self.sentence_list = []
        # self._augment_question(data_df)
        self._generate_data()
        
    def __getitem__(self, index):
        return self.sentence_list[index]
    
    def __len__(self):
        return len(self.sentence_list)

    def _augment_question(self, data_df):
        for _, row in data_df.iterrows():
            new_row = {}
            random_number = random.choice(range(0, len(data_df)))
            new_row['질문_1'] = row['질문_1'] + ' ' + data_df['질문_1'][random_number]
            new_row['질문_2'] = row['질문_2'] + ' ' + data_df['질문_2'][random_number]
            new_row['답변_1'] = row['답변_1'] + ' ' + data_df['답변_1'][random_number]
            new_row['답변_2'] = row['답변_2'] + ' ' + data_df['답변_2'][random_number]
            new_row['답변_3'] = row['답변_3'] + ' ' + data_df['답변_3'][random_number]
            new_row['답변_4'] = row['답변_4'] + ' ' + data_df['답변_4'][random_number]
            new_row['답변_5'] = row['답변_5'] + ' ' + data_df['답변_5'][random_number]
            df = pd.DataFrame(new_row,  index = [0])
            data_df = pd.concat([data_df, df], ignore_index=True)
        self.data_df = data_df.copy(deep=True)
        
    def _generate_data(self):
        for _, row in self.data_df.iterrows():
            for q_col in ['질문_1', '질문_2']:
                for a_col in ['답변_6', '답변_7']:
                    # 질문과 답변 쌍을 </s> token으로 연결
                    input_text = row[q_col] + tokenizer.eos_token + row[a_col] + tokenizer.eos_token
                    input_ids = tokenizer.encode(input_text, return_tensors='pt')
                    self.sentence_list.append(input_ids)
                    


In [17]:
# for name, module in model.named_children():
#     if name != "lm_head":
#         for param in module.parameters():
#             param.requires_grad = False
            

In [ ]:

# model.to(device) # 모델을 GPU단으로 이동

# 모델 학습 설정
optimizer = AdamW(model.parameters(), lr=CFG['LR'])
# optimizer = bnb.optim.Adam8bit(optimizer_grouped_parameters, lr=args.learning_rate)

scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.001)
# peft_model.config.use_cache = False 
# peft_model.train()
model.train()

# 모델 학습
for epoch in range(CFG['EPOCHS']):
    total_loss = 0
    dataset = HansolDataset(data_df = data_df, tokenizer=tokenizer)
    dataloader = DataLoader(dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True) # 미니 배치 형태로 데이터 갖추기
    progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for batch_idx, batch in progress_bar:
        # 데이터를 GPU단으로 이동
        batch = batch[0]
        # batch.unsqueeze(0)
        batch = batch.to(device)
        outputs = model(batch, labels=batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

        # 진행률 표시줄에 평균 손실 업데이트
        progress_bar.set_description(f"Epoch {epoch+1} - Avg Loss: {total_loss / (batch_idx+1):.4f}")

    # 에폭의 평균 손실을 출력
    print(f"Epoch {epoch+1}/{CFG['EPOCHS']}, Average Loss: {total_loss / len(dataloader)}")
    model.save_pretrained(f"./solar_bnb_{epoch}_epoch")
    tokenizer.save_pretrained(f"./solar_bnb_{epoch}_epoch")

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1 - Avg Loss: 0.2060: 100%|██████████| 2576/2576 [18:59<00:00,  2.26it/s]


Epoch 1/50, Average Loss: 0.20601882989466283


Epoch 2 - Avg Loss: 0.1699: 100%|██████████| 2576/2576 [19:06<00:00,  2.25it/s]


Epoch 2/50, Average Loss: 0.1698726827267164


Epoch 3 - Avg Loss: 0.1574: 100%|██████████| 2576/2576 [19:01<00:00,  2.26it/s]


...

Epoch 10 - Avg Loss: 0.1045: 100%|█████████| 2576/2576 [18:41<00:00,  2.30it/s]


Epoch 10/50, Average Loss: 0.10452674197247078


Epoch 11 - Avg Loss: 0.0997: 100%|█████████| 2576/2576 [18:41<00:00,  2.30it/s]


Epoch 11/50, Average Loss: 0.0997360258823064


Epoch 12 - Avg Loss: 0.0954:  94%|████████▍| 2432/2576 [17:38<00:58,  2.44it/s]